# EY 2025 Open Science Machine Learning & AI Challenge

In 2025, I participated in the 2025 EY Open Science challenge. The task was to create a model that uses satellite and building footprint data to predict urban heat island indices in NYC. The urban heat island (UHI) index is a measure of how hot a particular area feels to a human being on the ground relative to the mean. High UHIs are associated with worse health outcomes, and understanding the relationship between UHI and various urban structures is important to urban planning.

My model achieved an **R^2 of 0.973** on out-of-sample test data, placing in the top 50 of the final leaderboard. The challenge had **23,000 submissions globally.**

#### Setup Part 1: Import Modules

In [13]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import rioxarray as rxr
import rasterio
from rasterio.windows import from_bounds
from rasterio.warp import transform_bounds
from shapely.geometry import Point
from pyproj import Transformer

from scipy.ndimage import gaussian_filter, map_coordinates

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import geopandas as gpd
from shapely import wkt
from rtree import index

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


#### Setup Part 2: Create All Functions for Importing and Transforming

In [2]:
def generate_buffered_rasters(tiff_path, buffer_sizes=[450, 500, 550, 600, 650, 700, 800, 900, 1000, 1050, 1100, 1200, 1250]):
    """
    Reads a GeoTIFF, applies focal buffers with Gaussian filtering, and returns a dictionary of buffered rasters.

    Parameters:
    - tiff_path (str): Path to the input GeoTIFF file.
    - buffer_sizes (list): List of buffer distances in meters.

    Returns:
    - buffered_data (dict): Dictionary of buffered raster values for each band and buffer size.
    """

    data = rxr.open_rasterio(tiff_path)
    
    # Convert buffer sizes from meters to pixels (assuming 10m resolution per pixel)
    buffer_sizes_pixels = [size / 10 for size in buffer_sizes]  # Convert meters to pixel scale
    
    buffered_data = {}
    
    for band in data.coords["band"].values:
        buffered_data[band] = {}
        
        for size, sigma in zip(buffer_sizes, buffer_sizes_pixels):  
            # Apply Gaussian smoothing with sigma proportional to buffer size
            smoothed = gaussian_filter(data.sel(band=band, method='nearest'), sigma=sigma, mode="nearest")
            buffered_data[band][size] = smoothed

    print(f"Generated buffered rasters for {len(buffer_sizes)} buffer sizes across {len(data.coords['band'])} bands.")
    
    return buffered_data

def extract_values_from_rasters(buffered_data, csv_path, tiff_path):
    """
    Extracts buffered raster values at specified lat/lon locations from a CSV file.
    
    Parameters:
    - buffered_data (dict): Dictionary of buffered raster values.
    - transform (Affine): Raster transformation metadata for coordinate mapping.
    - csv_path (str): Path to the CSV file with 'Latitude' and 'Longitude' columns.

    Returns:
    - buffer_df (pd.DataFrame): DataFrame with extracted values mapped to lat/lon.
    """
    
    df = pd.read_csv(csv_path)
    latitudes = df["Latitude"].values
    longitudes = df["Longitude"].values
    
    with rasterio.open(tiff_path) as src:
        transform = src.transform

    # Convert lat/lon to raster pixel coordinates
    x_indices, y_indices = rasterio.transform.rowcol(transform, longitudes, latitudes)


    extracted_values = {size: {band: [] for band in buffered_data.keys()} for size in buffered_data[list(buffered_data.keys())[0]]}
    
    # Extract values using bilinear interpolation
    for band in buffered_data.keys():  
        for size in buffered_data[band]:  
            raster = buffered_data[band][size]
            values = map_coordinates(raster, [y_indices, x_indices], order=1)  # order=1 → bilinear
            extracted_values[size][band] = values

    # Create DataFrame and store extracted values
    buffer_df = pd.DataFrame()
    buffer_df["Latitude"] = latitudes
    buffer_df["Longitude"] = longitudes

    # Define a manual mapping of band numbers to their actual names
    band_mapping = {
        1: "B02",  2: "B03",  3: "B04",  4: "B05",  5: "B06",
        6: "B07",  7: "B08",  8: "B8A",  9: "B11", 10: "B12"
    }  # Modify if needed

# Assign extracted values to DataFrame with correct band names
    for size in buffered_data[list(buffered_data.keys())[0]]:  
        for band in buffered_data.keys():
            band_name = band_mapping.get(band, f"Band{band}")  # Use mapping or fallback
            column_name = f"{band_name}_{size}m"
            buffer_df[column_name] = extracted_values[size][band]

    return buffer_df

def map_satellite_data(tiff_path, csv_path):
    """
    Main function to extract buffered satellite data for specified coordinates.
    
    Parameters:
    - tiff_path (str): Path to the GeoTIFF file.
    - csv_path (str): Path to the CSV file containing coordinates.

    Returns:
    - buffer_df (pd.DataFrame): DataFrame with extracted satellite values at each buffer value.
    """
    
    buffered_data = generate_buffered_rasters(tiff_path)
    buffer_df = extract_values_from_rasters(buffered_data, csv_path, tiff_path)
    
    return buffer_df


def merge_data(buffer_df, ground_df):
    """
    Merges the buffered satellite data with the ground truth data.
    
    Parameters:
    - buffer_df (pd.DataFrame): DataFrame with buffered satellite values.
    - ground_df (pd.DataFrame): DataFrame with ground truth UHI index values.

    Returns:
    - merged_df (pd.DataFrame): Merged DataFrame with all features and target values.
    """
    
    merged_df = pd.merge(ground_df, buffer_df, on=["Latitude", "Longitude"])
    
    return merged_df

def compute_ndvi(df, nir_col, red_col):
    return (df[nir_col] - df[red_col]) / (df[nir_col] + df[red_col] + 1e-10)

def compute_ndbi(df, swir_col, nir_col):
    return (df[swir_col] - df[nir_col]) / (df[swir_col] + df[nir_col] + 1e-10)

def compute_evi(df, nir_col, red_col, blue_col):
    return 2.5 * (df[nir_col] - df[red_col]) / (df[nir_col] + 6 * df[red_col] - 7.5 * df[blue_col] + 1)

def compute_albedo(df, bands):
    weights = {'B02': 0.356, 'B04': 0.130, 'B05': 0.373, 'B06': 0.085, 'B07': 0.072}
    return sum(weights[band[:3]] * df[band] for band in bands if band[:3] in weights and band in df.columns)

def generate_landsat_lst_buffers(lst_array, buffer_sizes=[450, 500, 550, 600, 650, 700, 800, 900, 1000]):
    """
    Generates focal buffers for Landsat LST using Gaussian filtering.

    Parameters:
    - lst_array (numpy array): The LST raster array.
    - buffer_sizes (list): Buffer distances in meters.

    Returns:
    - buffered_lst (dict): Dictionary of buffered LST values for each buffer size.
    """
    buffer_sizes_pixels = [size / 100 for size in buffer_sizes]  # Convert meters to pixels (100m LST)
    
    buffered_lst = {}
    for size, sigma in zip(buffer_sizes, buffer_sizes_pixels):  
        buffered_lst[size] = gaussian_filter(lst_array, sigma=sigma, mode="nearest")
    
    return buffered_lst

def extract_and_merge_lst(buffered_lst, csv_path, tiff_path, uhi_data):

    # Open Landsat LST (100m) GeoTIFF
    with rasterio.open(tiff_path) as src:
        transform = src.transform  # Get geotransform
        lst_array = src.read(1)  # Read LST data (1st band)
        
        # Convert lat/lon to nearest raster indices
        x_indices, y_indices = rasterio.transform.rowcol(transform, uhi_data["Longitude"], uhi_data["Latitude"])
        
        # Ensure indices are within raster bounds
        x_indices = np.clip(x_indices, 0, lst_array.shape[1] - 1)
        y_indices = np.clip(y_indices, 0, lst_array.shape[0] - 1)

        # Extract LST values from nearest 100m pixel
        uhi_data["LST_Landsat"] = lst_array[y_indices, x_indices]

    # Extract buffered values
    for size in buffered_lst:
        raster = buffered_lst[size]
        
        # Extract buffered LST values using bilinear interpolation
        values = map_coordinates(raster, [y_indices, x_indices], order=1)  # order=1 → bilinear
        uhi_data[f"LST_Landsat_{size}m"] = values
    
    return uhi_data

## Import and Transform All Data

In [3]:
# import initial training data: latitude, longitude, uHI index, and date-time
ground_df = pd.read_csv("Training_data_uhi_index.csv")
satellite_data = map_satellite_data('S2.tiff', 'Training_data_uhi_index.csv')
uhi_data = merge_data(satellite_data, ground_df)
uhi_data.head()

Generated buffered rasters for 13 buffer sizes across 10 bands.


,Longitude,Latitude,datetime,UHI Index,B02_450m,B03_450m,B04_450m,B05_450m,B06_450m,B07_450m,...,B02_1250m,B03_1250m,B04_1250m,B05_1250m,B06_1250m,B07_1250m,B08_1250m,B8A_1250m,B11_1250m,B12_1250m
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,1024.753637,1100.867537,1065.042520,1183.392623,1383.586915,1483.534492,...,1113.083783,1225.088628,1223.513105,1408.657550,1765.125097,1914.794001,1889.849272,1965.490964,1785.894722,1486.500559
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,1024.753637,1100.867537,1065.042520,1183.392623,1383.586915,1483.534492,...,1113.083783,1225.088628,1223.513105,1408.657550,1765.125097,1914.794001,1889.849272,1965.490964,1785.894722,1486.500559
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,1020.947884,1096.392403,1059.205301,1176.426286,1373.596070,1472.639563,...,1112.816114,1224.873440,1223.112812,1408.380719,1765.159614,1914.956789,1890.080598,1965.751971,1785.680934,1486.078274
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,1016.552782,1091.238081,1052.364329,1168.330224,1362.923267,1461.230006,...,1113.169650,1225.248076,1223.397684,1408.606423,1764.879844,1914.517152,1889.636289,1965.272174,1785.591810,1486.221119
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,1012.181246,1086.111390,1045.562617,1160.281392,1352.306287,1449.877363,...,1113.523547,1225.624278,1223.684411,1408.836328,1764.612524,1914.092998,1889.208008,1964.809280,1785.513890,1486.371915


Now, we have the initial set of useful data: latitude, longitude, and the following bands for the buffer rasters of 450m - 1250m.

I chose the raster buffer sizes and bands based on trial and error and Shandas, et al (2019). 

The bands I selected were: 
- B02
- B03
- B04
- B05
- B06
- B07 
- B08
- B8A
- B11
- B12

#### Step 4: Transform Data - Adding Indices

To add additional information to this dataset, I create 5 indices from the bands: 
1) Normalized Difference Vegetation Index, which quantifies vegetation
2) Normalized Difference Built-up Index, which measures how muc an area is built up
3) Enhanced Vegetation Index, another way to quantify vegetation
4) Albedo, a measure of the reflectiveness of a surface

In [4]:
buffer_sizes = [450, 500, 550, 600, 650, 700, 800, 900, 1000]

for buffer in buffer_sizes:
    uhi_data[f"NDVI_{buffer}"] = compute_ndvi(uhi_data, f"B08_{buffer}m", f"B04_{buffer}m")
    uhi_data[f"NDBI_{buffer}"] = compute_ndbi(uhi_data, f"B11_{buffer}m", f"B08_{buffer}m")
    uhi_data[f"EVI_{buffer}"] = compute_evi(uhi_data, f"B08_{buffer}m", f"B04_{buffer}m", f"B02_{buffer}m")
    uhi_data[f"Albedo_{buffer}"] = compute_albedo(uhi_data, [f"B02_{buffer}m", f"B04_{buffer}m", f"B05_{buffer}m", f"B06_{buffer}m", f"B07_{buffer}m"])

columns_to_check = uhi_data.columns[3:]
for col in columns_to_check:
    uhi_data[col] = uhi_data[col].apply(lambda x: tuple(x) if isinstance(x, np.ndarray) and x.ndim > 0 else x)
uhi_data = uhi_data.drop_duplicates(subset=columns_to_check, keep='first')

#### Step 5: Adding Landsat Data

In [5]:
filename = "Landsat_LST.tiff"
lst_data = rxr.open_rasterio(filename)  # Load LST

# Ensure CRS is set correctly
lst_data = lst_data.rio.write_crs("EPSG:4326")

with rasterio.open("Landsat_LST.tiff") as src:
    transform = src.transform  # Get transform for coordinate conversion
    lst_array = src.read(1)  # Read LST data (1st band)
    
    # Convert lat/lon to nearest raster indices
    x_indices, y_indices = rasterio.transform.rowcol(transform, uhi_data["Longitude"], uhi_data["Latitude"])
    
    # Ensure indices are within bounds
    x_indices = np.clip(x_indices, 0, lst_array.shape[1] - 1)
    y_indices = np.clip(y_indices, 0, lst_array.shape[0] - 1)

    # Extract LST values from nearest 100m pixel
    uhi_data["LST_Landsat"] = lst_array[y_indices, x_indices]

buffered_lst = generate_landsat_lst_buffers(lst_array)
uhi_data = extract_and_merge_lst(buffered_lst, "Training_data_uhi_index.csv", "Landsat_LST.tiff", uhi_data)
uhi_data.head()

,Longitude,Latitude,datetime,UHI Index,B02_450m,B03_450m,B04_450m,B05_450m,B06_450m,B07_450m,...,LST_Landsat,LST_Landsat_450m,LST_Landsat_500m,LST_Landsat_550m,LST_Landsat_600m,LST_Landsat_650m,LST_Landsat_700m,LST_Landsat_800m,LST_Landsat_900m,LST_Landsat_1000m
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,1024.753637,1100.867537,1065.042520,1183.392623,1383.586915,1483.534492,...,32.766171,31.989961,31.753957,31.548813,31.373406,31.226099,31.105029,30.933838,30.844487,30.822467
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,1020.947884,1096.392403,1059.205301,1176.426286,1373.596070,1472.639563,...,32.340628,31.614588,31.437409,31.287579,31.162986,31.061527,30.981157,30.876225,30.834774,30.845613
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,1016.552782,1091.238081,1052.364329,1168.330224,1362.923267,1461.230006,...,32.340628,31.614588,31.437409,31.287579,31.162986,31.061527,30.981157,30.876225,30.834774,30.845613
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,1012.181246,1086.111390,1045.562617,1160.281392,1352.306287,1449.877363,...,30.171894,30.350946,30.274358,30.212952,30.166106,30.133271,30.113896,30.113296,30.159188,30.245414
6,-73.909312,40.812710,24-07-2021 15:53,1.015143,1008.462790,1081.729943,1039.848712,1153.456857,1342.482766,1439.154574,...,30.171894,30.350946,30.274358,30.212952,30.166106,30.133271,30.113896,30.113296,30.159188,30.245414


#### Step 6: Adding Building Density Data

In [6]:
buildings = gpd.read_file("Building_Footprint.kml")

buildings = buildings.to_crs("EPSG:32618")
buildings.head()
uhi_data["geometry"] = uhi_data.apply(lambda row: Point(row["Longitude"], row["Latitude"]), axis=1)
uhi_data = gpd.GeoDataFrame(uhi_data, geometry="geometry", crs="EPSG:4326")

uhi_data = uhi_data.to_crs(buildings.crs)

buffer_sizes = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000]

for buffer_size in buffer_sizes:
    uhi_data[f"building_density_{buffer_size}m"] = uhi_data["geometry"].apply(
        lambda x: buildings[buildings.geometry.intersects(x.buffer(buffer_size))].geometry.area.sum()
    )

#### Step 7: Adding Building Height Data

This data comes from the one additional dataset I chose to use in order to gather building heights (which the original data footprint did not include). I gathered it from the NYC Open Data portal, linked [here](https://data.cityofnewyork.us/City-Government/Building-Footprints/5zhs-2jue/about_data).

In [7]:
building_heights = pd.read_csv("Building_Footprints.csv")
building_heights = building_heights[building_heights['CNSTRCT_YR'] < 2022]
building_heights["geometry"] = building_heights["the_geom"].apply(wkt.loads)
gdf = gpd.GeoDataFrame(building_heights[["geometry", "HEIGHTROOF"]], geometry="geometry", crs="EPSG:4326")

# convert the uhi data into a geodataframe for merging purposes
uhi_gdf = gpd.GeoDataFrame(
    uhi_data,
    geometry=gpd.points_from_xy(uhi_data["Longitude"], uhi_data["Latitude"]),
    crs="EPSG:4326"
)

In [8]:
search_radius = 500  # Radius in meters

# Convert to projected CRS (EPSG:3857) for distance calculations
uhi_gdf = uhi_gdf.to_crs(epsg=3857)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
gdf = gdf.to_crs(epsg=3857)

# Build R-tree spatial index for fast lookup
building_index = index.Index()
for i, geom in enumerate(gdf.geometry):
    building_index.insert(i, geom.bounds)

def get_building_stats(point):
    nearby_indices = list(building_index.intersection(point.buffer(search_radius).bounds))
    nearby_buildings = gdf.iloc[nearby_indices]

    if not nearby_buildings.empty:
        return pd.Series({
            "avg_height": nearby_buildings["HEIGHTROOF"].mean(),
            "max_height": nearby_buildings["HEIGHTROOF"].max(),
            "num_buildings": len(nearby_buildings)
        })
    else:
        return pd.Series({"avg_height": None, "max_height": None, "num_buildings": 0})

# Apply the optimized function to all UHI points
uhi_gdf[["avg_height_500m", "max_height_500m", "num_buildings_500m"]] = uhi_gdf["geometry"].apply(get_building_stats)

uhi_data = uhi_gdf.sjoin_nearest(gdf[["geometry", "HEIGHTROOF"]], how="left", distance_col="distance_to_building")
uhi_data.head()

,Longitude,Latitude,datetime,UHI Index,B02_450m,B03_450m,B04_450m,B05_450m,B06_450m,B07_450m,...,building_density_700m,building_density_800m,building_density_900m,building_density_1000m,avg_height_500m,max_height_500m,num_buildings_500m,index_right,HEIGHTROOF,distance_to_building
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,1024.753637,1100.867537,1065.042520,1183.392623,1383.586915,1483.534492,...,500694.520091,619970.534374,784405.309617,990085.767677,40.096943,208.343052,360.0,706537,65.10,17.180988
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,1020.947884,1096.392403,1059.205301,1176.426286,1373.596070,1472.639563,...,496611.656620,623171.383644,792792.965140,990577.919304,39.415486,208.343052,359.0,706537,65.10,18.253962
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,1016.552782,1091.238081,1052.364329,1168.330224,1362.923267,1461.230006,...,505404.855287,624331.972110,790488.393024,990771.620216,38.988153,208.343052,363.0,706537,65.10,18.274486
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,1012.181246,1086.111390,1045.562617,1160.281392,1352.306287,1449.877363,...,515003.597141,622662.282688,794319.763560,985447.972125,38.931341,208.343052,365.0,706537,65.10,17.548956
6,-73.909312,40.812710,24-07-2021 15:53,1.015143,1008.462790,1081.729943,1039.848712,1153.456857,1342.482766,1439.154574,...,483298.840723,623027.368137,796284.038893,978535.871089,38.846073,208.343052,376.0,469932,12.83,18.014776


#### Step 8: Remove Features I Don't Want in Training

In [9]:
uhi_data = uhi_data.drop(columns=["Latitude", "Longitude", "datetime", "geometry", "index_right"])
uhi_data.head()

,UHI Index,B02_450m,B03_450m,B04_450m,B05_450m,B06_450m,B07_450m,B08_450m,B8A_450m,B11_450m,...,building_density_600m,building_density_700m,building_density_800m,building_density_900m,building_density_1000m,avg_height_500m,max_height_500m,num_buildings_500m,HEIGHTROOF,distance_to_building
0,1.030289,1024.753637,1100.867537,1065.042520,1183.392623,1383.586915,1483.534492,1448.231654,1501.318936,1375.428225,...,365848.350054,500694.520091,619970.534374,784405.309617,990085.767677,40.096943,208.343052,360.0,65.10,17.180988
2,1.023798,1020.947884,1096.392403,1059.205301,1176.426286,1373.596070,1472.639563,1437.221300,1489.910691,1363.606527,...,365158.719517,496611.656620,623171.383644,792792.965140,990577.919304,39.415486,208.343052,359.0,65.10,18.253962
3,1.023798,1016.552782,1091.238081,1052.364329,1168.330224,1362.923267,1461.230006,1425.663835,1477.936690,1350.477420,...,365115.964480,505404.855287,624331.972110,790488.393024,990771.620216,38.988153,208.343052,363.0,65.10,18.274486
4,1.021634,1012.181246,1086.111390,1045.562617,1160.281392,1352.306287,1449.877363,1414.163407,1466.020896,1337.435246,...,367216.913703,515003.597141,622662.282688,794319.763560,985447.972125,38.931341,208.343052,365.0,65.10,17.548956
6,1.015143,1008.462790,1081.729943,1039.848712,1153.456857,1342.482766,1439.154574,1403.330666,1454.787334,1325.870053,...,368240.347680,483298.840723,623027.368137,796284.038893,978535.871089,38.846073,208.343052,376.0,12.83,18.014776


## Training the Model

In [10]:
X = uhi_data.drop(columns=['UHI Index']).values
y = uhi_data ['UHI Index'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=123)

In [11]:
from xgboost import XGBRegressor

# Train XGBoost model
xgb = XGBRegressor(n_estimators=300, learning_rate=0.05, max_depth=10, random_state=42)
xgb.fit(X_train, y_train)

# Predict & Evaluate
y_pred_xgb = xgb.predict(X_test)
r2_xgb = r2_score(y_test, y_pred_xgb)
print(f"R² with XGBoost: {r2_xgb:.4f}")

R² with XGBoost: 0.9585


#### Final Step: Retrain the Model on All Data

By retraining the model on all data before creating the predictions, I am able to boost my final prediction R^2 above the out-of-sample.

In [14]:
X = uhi_data.drop(columns=["UHI Index"]).values
y = uhi_data['UHI Index'].values
sc = StandardScaler()
X = sc.fit_transform(X)

xgb = XGBRegressor(n_estimators=300, learning_rate=0.05, max_depth=9, random_state=42)
final_model = xgb.fit(X, y)

## **CREATE SUBMISSION**

First, I need to recreate the test data with all of the features generated.

In [ ]:
test_file = pd.read_csv('Submission_template.csv')
val_data = map_satellite_data('S2.tiff', 'Submission_template.csv')
buffer_sizes = [450, 500, 550, 600, 650, 700, 800, 900, 1000]

for buffer in buffer_sizes:
    val_data[f"NDVI_{buffer}"] = compute_ndvi(val_data, f"B08_{buffer}m", f"B04_{buffer}m")
    val_data[f"NDBI_{buffer}"] = compute_ndbi(val_data, f"B11_{buffer}m", f"B08_{buffer}m")
    val_data[f"EVI_{buffer}"] = compute_evi(val_data, f"B08_{buffer}m", f"B04_{buffer}m", f"B02_{buffer}m")
    val_data[f"Albedo_{buffer}"] = compute_albedo(val_data, [f"B02_{buffer}m", f"B04_{buffer}m", f"B05_{buffer}m", f"B06_{buffer}m", f"B07_{buffer}m"])

Generated buffered rasters for 13 buffer sizes across 10 bands.


Add Landsat data with buffers, building density, and building height data.

In [ ]:
# landsat data

filename = "Landsat_LST.tiff"
lst_data = rxr.open_rasterio(filename)  

lst_data = lst_data.rio.write_crs("EPSG:4326")

with rasterio.open("Landsat_LST.tiff") as src:
    transform = src.transform  
    lst_array = src.read(1)  
    
    x_indices, y_indices = rasterio.transform.rowcol(transform, val_data["Longitude"], val_data["Latitude"])
    
    x_indices = np.clip(x_indices, 0, lst_array.shape[1] - 1)
    y_indices = np.clip(y_indices, 0, lst_array.shape[0] - 1)

    val_data["LST_Landsat"] = lst_array[y_indices, x_indices]

buffered_lst = generate_landsat_lst_buffers(lst_array)

val_data = extract_and_merge_lst(buffered_lst, "Submission_template.csv", "Landsat_LST.tiff", val_data)

# add building density features
val_data["geometry"] = val_data.apply(lambda row: Point(row["Longitude"], row["Latitude"]), axis=1)
val_data = gpd.GeoDataFrame(val_data, geometry="geometry", crs="EPSG:4326")
val_data = val_data.to_crs(buildings.crs)

buffer_sizes = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000]

for buffer_size in buffer_sizes:
    val_data[f"building_density_{buffer_size}m"] = val_data["geometry"].apply(
        lambda x: buildings[buildings.geometry.intersects(x.buffer(buffer_size))].geometry.area.sum()
    )

In [ ]:
# add building height features

building_heights = pd.read_csv("Building_Footprints.csv")
building_heights = building_heights[building_heights['CNSTRCT_YR'] < 2022]

building_heights["geometry"] = building_heights["the_geom"].apply(wkt.loads)  # Convert WKT to geometry

gdf = gpd.GeoDataFrame(building_heights[["geometry", "HEIGHTROOF"]], geometry="geometry")

val_gdf = gpd.GeoDataFrame(val_data, geometry=gpd.points_from_xy(val_data["Longitude"], val_data["Latitude"]), crs="EPSG:4326")

search_radius = 500  

val_gdf = val_gdf.to_crs(epsg=3857)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
gdf = gdf.to_crs(epsg=3857)

building_index = index.Index()
for i, geom in enumerate(gdf.geometry):
    building_index.insert(i, geom.bounds)

def get_building_stats(point):
    nearby_indices = list(building_index.intersection(point.buffer(search_radius).bounds))
    nearby_buildings = gdf.iloc[nearby_indices]

    if not nearby_buildings.empty:
        return pd.Series({
            "avg_height": nearby_buildings["HEIGHTROOF"].mean(),
            "max_height": nearby_buildings["HEIGHTROOF"].max(),
            "num_buildings": len(nearby_buildings)
        })
    else:
        return pd.Series({"avg_height": None, "max_height": None, "num_buildings": 0})

# Apply the optimized function to all UHI points
val_gdf[["avg_height_500m", "max_height_500m", "num_buildings_500m"]] = val_gdf["geometry"].apply(get_building_stats)

val_data = val_gdf.sjoin_nearest(gdf[["geometry", "HEIGHTROOF"]], how="left", distance_col="distance_to_building")
val_data.head()

,Latitude,Longitude,B02_450m,B03_450m,B04_450m,B05_450m,B06_450m,B07_450m,B08_450m,B8A_450m,...,building_density_700m,building_density_800m,building_density_900m,building_density_1000m,avg_height_500m,max_height_500m,num_buildings_500m,index_right,HEIGHTROOF,distance_to_building
0,40.788763,-73.971665,1504.939598,1642.995149,1739.445082,1952.773795,2250.764985,2376.622815,2323.140299,2409.930613,...,712875.633507,807243.198834,880245.397641,956067.936203,71.135657,339.758242,807.0,942446,62.000000,21.078770
1,40.788875,-73.971928,1494.856374,1633.415562,1728.387802,1942.397263,2247.169629,2375.219836,2322.161841,2409.141746,...,716036.107188,807243.198834,887976.668907,967377.849327,72.298800,339.758242,827.0,942446,62.000000,21.015439
2,40.789080,-73.967080,1547.080817,1681.453239,1782.929608,1995.191488,2270.027670,2389.962843,2336.913316,2421.998148,...,446738.870121,568730.831418,708153.603209,981122.105507,70.084045,339.758242,516.0,529480,235.437245,1.334386
3,40.789082,-73.972550,1476.299696,1615.715764,1707.587393,1922.899732,2240.225168,2372.306598,2320.142142,2407.379957,...,717777.644968,796459.659417,887263.753150,986609.310191,71.800322,339.758242,857.0,954872,93.110000,2.230911
4,40.787953,-73.969697,1552.376690,1687.174664,1788.993895,1999.801673,2268.576682,2385.426943,2330.684674,2415.982289,...,595208.633145,756287.297101,829802.336755,976210.864819,69.687768,339.758242,746.0,353416,60.280000,12.705511


In [ ]:
# drop unnecessary columns

val_data = val_data.drop(columns=["Latitude", "Longitude", "geometry", "index_right"])

In [ ]:
submission_val_data = val_data.values
transformed_submission_data = sc.transform(submission_val_data)

final_predictions = final_model.predict(transformed_submission_data)
final_prediction_series = pd.Series(final_predictions)

submission_df = pd.DataFrame({'Longitude':test_file['Longitude'].values, 'Latitude':test_file['Latitude'].values, 'UHI Index':final_prediction_series.values})
# submission_df.to_csv("submission.csv",index = False)

### Final Thoughts and Discussions

Overall, I learned a lot from this project, and I am proud of the model I built. However, it still has some real issues with interpretability. For example, my final model used a whopping 196 features, and many of them were averages of densities or satellite bands at varying distances. This would **not** be helpful with urban planning, but I felt I had to have such a complex model to get into the high R-squared zone. 

I can't wait to see how the winners approached it and learn more about how to get an effective R-squared without the lack of interpretability that my model may have suffered from!